In [1]:
import pandas as pd
import pandas_ta as ta
import numpy as np

path = '../data/ADAUSDT_4h.csv'
symbol = path.split('/')[-1].split('_')[0].replace('USDT', '')
df = pd.read_csv(path)
df.head()

,time,open,high,low,close,volume
0,2020-01-31 08:00:00,0.05450,0.05559,0.05374,0.05414,70075549.0
1,2020-01-31 12:00:00,0.05413,0.05419,0.05209,0.05371,67903775.0
2,2020-01-31 16:00:00,0.05367,0.05386,0.05264,0.05299,49808861.0
3,2020-01-31 20:00:00,0.05302,0.05458,0.05302,0.05387,56844669.0
4,2020-02-01 00:00:00,0.05383,0.05598,0.05360,0.05482,61545873.0


In [2]:
temp = df.ta.bbands()
df['BBANDS_U'] = temp['BBU_5_2.0']
df['BBANDS_L'] = temp['BBL_5_2.0']
df['BBANDS_M'] = temp['BBM_5_2.0']
temp = df.ta.cdl_pattern(name="closingmarubozu")
df['MARUBOZU'] = temp['CDL_CLOSINGMARUBOZU']
#singal_buy = MARUBOZU == 100 and close > DONCH_U 
#singal_sell = MARUBOZU == -100 and close < DONCH_L
df['Signal_Buy'] = np.where((df['MARUBOZU'] == 100) & (df['close'] < df['BBANDS_U']), 1, 0)
df['Signal_Sell'] = np.where((df['MARUBOZU'] == -100) & (df['close'] < df['BBANDS_L']), 1, 0)
df['Signal'] = df['Signal_Buy'].replace(1,2) + df['Signal_Sell']
df['ATR'] = df.ta.atr()
df['ADX'] = df.ta.adx()['ADX_14']
#help(ta.adx)

In [3]:
df['Signal'].value_counts()

Signal
0    6156
2     344
Name: count, dtype: int64

In [4]:
df.ta.bbands()

,BBL_5_2.0,BBM_5_2.0,BBU_5_2.0,BBB_5_2.0,BBP_5_2.0
0,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN
4,0.052716,0.053906,0.055096,4.413590,0.884164
...,...,...,...,...,...
6495,0.345202,0.351220,0.357238,3.426821,0.024746
6496,0.344985,0.351060,0.357135,3.460946,0.429218
6497,0.345008,0.350380,0.355752,3.066173,0.511170
6498,0.343971,0.349200,0.354429,2.994931,0.289641


In [5]:
help(ta.donchian)

Help on function donchian in module pandas_ta.volatility.donchian:

donchian(high, low, lower_length=None, upper_length=None, offset=None, **kwargs)
    Donchian Channels (DC)
    
    Donchian Channels are used to measure volatility, similar to
    Bollinger Bands and Keltner Channels.
    
    Sources:
        https://www.tradingview.com/wiki/Donchian_Channels_(DC)
    
    Calculation:
        Default Inputs:
            lower_length=upper_length=20
        LOWER = low.rolling(lower_length).min()
        UPPER = high.rolling(upper_length).max()
        MID = 0.5 * (LOWER + UPPER)
    
    Args:
        high (pd.Series): Series of 'high's
        low (pd.Series): Series of 'low's
        lower_length (int): The short period. Default: 20
        upper_length (int): The short period. Default: 20
        offset (int): How many periods to offset the result. Default: 0
    
    Kwargs:
        fillna (value, optional): pd.DataFrame.fillna(value)
        fill_method (value, optional): Type

In [6]:
#dropna
df = df.dropna()
df.head()

,time,open,high,low,close,volume,BBANDS_U,BBANDS_L,BBANDS_M,MARUBOZU,Signal_Buy,Signal_Sell,Signal,ATR,ADX
27,2020-02-04 20:00:00,0.05588,0.05641,0.05567,0.05601,24377405.0,0.056577,0.055239,0.055908,0.0,0,0,0,0.001721,15.921279
28,2020-02-05 00:00:00,0.05605,0.05639,0.05572,0.05609,25862609.0,0.056522,0.055250,0.055886,0.0,0,0,0,0.001646,16.140291
29,2020-02-05 04:00:00,0.05609,0.05780,0.05605,0.05721,34970817.0,0.057229,0.055315,0.056272,0.0,0,0,0,0.001653,15.098556
30,2020-02-05 08:00:00,0.05723,0.05872,0.05686,0.05872,47694948.0,0.058941,0.054623,0.056782,100.0,1,0,2,0.001668,15.364193
31,2020-02-05 12:00:00,0.05872,0.06028,0.05831,0.06005,88378599.0,0.060742,0.054490,0.057616,0.0,0,0,0,0.001690,17.079056


In [7]:
#set time to index

df['time'] = pd.to_datetime(df['time'])
df = df.set_index('time')
df.head()

C:\Users\hiban\AppData\Local\Temp\ipykernel_17384\1852176384.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['time'] = pd.to_datetime(df['time'])


,open,high,low,close,volume,BBANDS_U,BBANDS_L,BBANDS_M,MARUBOZU,Signal_Buy,Signal_Sell,Signal,ATR,ADX
time,,,,,,,,,,,,,,
2020-02-04 20:00:00,0.05588,0.05641,0.05567,0.05601,24377405.0,0.056577,0.055239,0.055908,0.0,0,0,0,0.001721,15.921279
2020-02-05 00:00:00,0.05605,0.05639,0.05572,0.05609,25862609.0,0.056522,0.055250,0.055886,0.0,0,0,0,0.001646,16.140291
2020-02-05 04:00:00,0.05609,0.05780,0.05605,0.05721,34970817.0,0.057229,0.055315,0.056272,0.0,0,0,0,0.001653,15.098556
2020-02-05 08:00:00,0.05723,0.05872,0.05686,0.05872,47694948.0,0.058941,0.054623,0.056782,100.0,1,0,2,0.001668,15.364193
2020-02-05 12:00:00,0.05872,0.06028,0.05831,0.06005,88378599.0,0.060742,0.054490,0.057616,0.0,0,0,0,0.001690,17.079056


In [8]:
#rename columns
df.rename(columns={'open': 'Open', 'high': 'High', 'low': 'Low', 'close': 'Close', 'volume': 'Volume'}, inplace=True)

In [9]:
df['Signal'].value_counts()

Signal
0    6130
2     343
Name: count, dtype: int64

In [10]:
from backtesting import Strategy
from backtesting import Backtest

def SIGNAL(df,col_name):
    return df[col_name]

class Scalping_Strategy(Strategy):
    atr_sl_rate = 1.3
    TPSL_rate = 1.5
    save_size_rate = 0.125
    init_cash = 350
    profit_target = 500
    size_multiplier_rate = 0.5

    def init(self):
        super().init()
        self.signal = self.I(SIGNAL, self.data, 'Signal')

    def next(self):
        super().next()

        slatr = self.atr_sl_rate * self.data.ATR[-1]
        size_multiplier = self.size_multiplier_rate if self.equity >= self.profit_target else 1.0

        if self.signal == 2:
            sl_price = self.data.Close[-1] - slatr
            tp_price = self.data.Close[-1] + self.TPSL_rate * slatr
            size = min(self.save_size_rate * self.equity / slatr, self.init_cash / slatr) * size_multiplier
            if self.equity > self.init_cash * 1.5:
                self.buy(sl=sl_price, tp=tp_price, size=int(size))
            else:
                self.buy(sl=sl_price, tp=tp_price)

        if self.signal == 1:
            sl_price = self.data.Close[-1] + slatr
            tp_price = self.data.Close[-1] - self.TPSL_rate * slatr
            size = min(self.save_size_rate * self.equity / slatr, self.init_cash / slatr) * size_multiplier
            if self.equity >= self.init_cash * 1.5:
                self.sell(sl=sl_price, tp=tp_price, size=int(size))
            else:
                self.sell(sl=sl_price, tp=tp_price)
            
cash = 350
bt = Backtest(df, Scalping_Strategy, cash=cash, commission=.00002, margin=.05)
stat = bt.run()
stat

Start                     2020-02-04 20:00:00
End                       2023-01-18 12:00:00
Duration                   1078 days 16:00:00
Exposure Time [%]                    1.436737
Equity Final [$]                          0.0
Equity Peak [$]                    910.567602
Return [%]                             -100.0
Buy & Hold Return [%]              504.177825
Return (Ann.) [%]                         0.0
Volatility (Ann.) [%]         13607344.980094
Sharpe Ratio                              0.0
Sortino Ratio                             0.0
Calmar Ratio                              0.0
Max. Drawdown [%]                      -100.0
Avg. Drawdown [%]                  -28.126087
Max. Drawdown Duration     1064 days 16:00:00
Avg. Drawdown Duration      178 days 08:00:00
# Trades                                   14
Win Rate [%]                        28.571429
Best Trade [%]                        5.53711
Worst Trade [%]                     -6.986023
Avg. Trade [%]                    

In [11]:
import plotly.graph_objects as go

fig = go.Figure(data=[go.Scatter(x=stat['_equity_curve'].index, y=stat['_equity_curve']['Equity'])])
fig.show()

In [12]:
%%time
stats = bt.optimize(atr_sl_rate = np.arange(0.3, 2.6, 0.2).tolist(),
                    TPSL_rate = np.arange(0.3, 2.6, 0.2).tolist(),
                    size_multiplier_rate = np.arange(0.1, 0.5, 0.1).tolist(),
                    maximize='Equity Final [$]')
stats

C:\Users\hiban\AppData\Roaming\Python\Python311\site-packages\backtesting\backtesting.py:1488: UserWarning:

Searching for best of 576 configurations.



  0%|          | 0/12 [00:00<?, ?it/s]

C:\Users\hiban\AppData\Roaming\Python\Python311\site-packages\backtesting\_stats.py:114: RuntimeWarning:

overflow encountered in scalar power

C:\Users\hiban\AppData\Roaming\Python\Python311\site-packages\backtesting\_stats.py:114: RuntimeWarning:

overflow encountered in scalar power

C:\Users\hiban\AppData\Roaming\Python\Python311\site-packages\backtesting\_stats.py:114: RuntimeWarning:

overflow encountered in scalar power

C:\Users\hiban\AppData\Roaming\Python\Python311\site-packages\backtesting\_stats.py:114: RuntimeWarning:

overflow encountered in scalar power

C:\Users\hiban\AppData\Roaming\Python\Python311\site-packages\backtesting\_stats.py:114: RuntimeWarning:

overflow encountered in scalar power

C:\Users\hiban\AppData\Roaming\Python\Python311\site-packages\backtesting\_stats.py:114: RuntimeWarning:

overflow encountered in scalar power



CPU times: total: 58.3 s
Wall time: 58.7 s


Start                     2020-02-04 20:00:00
End                       2023-01-18 12:00:00
Duration                   1078 days 16:00:00
Exposure Time [%]                   78.294454
Equity Final [$]                 11718.678607
Equity Peak [$]                  16721.080395
Return [%]                        3248.193888
Buy & Hold Return [%]              504.177825
Return (Ann.) [%]                  227.589149
Volatility (Ann.) [%]              808.594138
Sharpe Ratio                         0.281463
Sortino Ratio                        4.960564
Calmar Ratio                         3.088875
Max. Drawdown [%]                  -73.680269
Avg. Drawdown [%]                   -7.519407
Max. Drawdown Duration      503 days 04:00:00
Avg. Drawdown Duration       14 days 06:00:00
# Trades                                  343
Win Rate [%]                        35.568513
Best Trade [%]                      65.057161
Worst Trade [%]                    -34.611592
Avg. Trade [%]                    

In [13]:
stats.keys()

Index(['Start', 'End', 'Duration', 'Exposure Time [%]', 'Equity Final [$]',
       'Equity Peak [$]', 'Return [%]', 'Buy & Hold Return [%]',
       'Return (Ann.) [%]', 'Volatility (Ann.) [%]', 'Sharpe Ratio',
       'Sortino Ratio', 'Calmar Ratio', 'Max. Drawdown [%]',
       'Avg. Drawdown [%]', 'Max. Drawdown Duration', 'Avg. Drawdown Duration',
       '# Trades', 'Win Rate [%]', 'Best Trade [%]', 'Worst Trade [%]',
       'Avg. Trade [%]', 'Max. Trade Duration', 'Avg. Trade Duration',
       'Profit Factor', 'Expectancy [%]', 'SQN', '_strategy', '_equity_curve',
       '_trades'],
      dtype='object')

In [14]:
stats._strategy

<Strategy Scalping_Strategy(atr_sl_rate=2.5,TPSL_rate=2.5,size_multiplier_rate=0.4)>

In [15]:
import plotly.graph_objects as go

fig = go.Figure(data=[go.Scatter(x=stats['_equity_curve'].index, y=stats['_equity_curve']['Equity'])])
fig.update_layout(title='Profit | '+symbol+' | '+ str(cash), yaxis_title='USDT', xaxis_title='Date')
fig.show()